In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# Array Magic 📡📡🪄

## 🎯 Learning Goals

By the end of this notebook, you will be able to:  
- **Understand** electronic beam steering: how **phase shifts** across elements steer the beam quickly, without moving parts. 
- **Describe** how **phase**, **wavelength**, and **element spacing** determine **beam direction** and **nulls**.  



# Directing Wave Energy With Precision and Speed 🌊🎯

## **Steering the Beam Without Moving the Array** 📡

So far, we’ve sent our waves generated by the bouys rocking in sync straight ahead in a tidy beam. But what if we want to look to the side or **scan the scene** from different directions? How do we **direct the wave energy** somewhere else?

Think of a flashlight. 🔦 It concentrates light into a tight spot—just like our buoy array concentrates wave energy into a focused beam. To explore a dark room, you’d simply **turn the flashlight** and aim it in different directions. We could do the same here: **rotate the whole array** so the beam points where we want. This works, but it comes with trade-offs:

- **Slow to move:** mechanical parts take time to start, stop, and settle. 🐌  
- **Fussy to control:** precise pointing needs careful alignment. 🎯  
- **Wear and tear:** moving mechanisms add bulk, friction, and maintenance. ⚙️  

Instead, let's try something **more elegant**:  

Rather than rocking all the buoys **in sync**, we introduce a **small delay** between them.  

1. We start rocking the **rightmost buoy** first. 🛟➡️  
2. A moment later, we start rocking the **next one**. 🛟⏱️  
3. This continues **progressively across the array**, until we reach the last buoy. 🛟🛟🛟🌊  

Now, every buoy still generates **the same wave**—with the **same amplitude and wavelength**—but at any given instant, the **phase** of each individual wave **is slightly different**. 🔄  

---

### **How Does This Affect the Beam Pattern?**  

Remember: when waves combine, **phase is crucial!**  

- **Where two crests meet**, the waves **reinforce each other**, creating a strong peak. 📈  
- **Where a crest and a trough meet**, they **cancel out**, leaving no wave at all. ❌  

Since we’ve **intentionally shifted the phase** of each buoy’s wave, their **combination will now form a beam in a new direction**! 📡✨  

This is the fundamental principle behind **electronic beam steering** in phased arrays. 🚀  

---

### **Try It Yourself!**  

Experiment with **phase delays** between neighboring buoys. Adjust the **phase shift** and observe how the **beam direction changes**. 🔄🎯  

In [ ]:
def build_beam_steering_sim():
    import io
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, IntSlider, Play, jslink, HBox, VBox, Layout, Label, Image as WImage, Button, Dropdown
    )
    from IPython.display import display

    # -----------------------
    # Fixed parameters (aligned with buoy sim)
    # -----------------------
    AMPLITUDE     = 1.0
    WAVE_SPEED    = 5.0        # m/s  (f = v / λ)
    GRID_SIZE     = 100        # Ny = Nx = GRID_SIZE
    DT            = 0.05       # s per frame
    FRAMES        = 200
    INTERVAL_MS   = 40
    SPACE_EXTENT  = 100.0      # x in [-E, E], y in [0, 2E]
    DPI           = 96

    # -----------------------
    # Widgets (layout & behavior matched to buoy sim)
    # -----------------------
    phase_cycles = FloatSlider(
        value=0.0, min=-0.5, max=0.5, step=0.01,
        description="Phase/elem [cycles]", continuous_update=False
    )
    phase_cycles.style.description_width = 'initial'

    wavelength = FloatSlider(
        value=10.0, min=5.0, max=20.0, step=0.5,
        description="Wavelength λ [m]", continuous_update=False
    )
    wavelength.style.description_width = 'initial'

    n_elements = IntSlider(
        value=6, min=1, max=12, step=1,
        description="Elements N", continuous_update=False
    )
    n_elements.style.description_width = 'initial'

    play = Play(value=0, min=0, max=FRAMES, step=1, interval=INTERVAL_MS)
    t_slider = IntSlider(value=0, min=0, max=FRAMES, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')
    
    cmap_dd = Dropdown(options=['jet', 'viridis', 'plasma', 'gray', 'coolwarm'], 
                       value='jet', description='Colormap:', 
                       style={'description_width':'initial'})

    readout = Label()
    controls = VBox([
        HBox([wavelength, n_elements, phase_cycles, cmap_dd],
             layout=Layout(align_items='center', column_gap='15px')),
        HBox([play, t_slider, reset_btn, save_btn],
             layout=Layout(align_items='center', column_gap='15px'))
    ])

    # -----------------------
    # Spatial grid (fixed)
    # -----------------------
    Nx = Ny = GRID_SIZE
    x = np.linspace(-SPACE_EXTENT, SPACE_EXTENT, Nx)
    y = np.linspace(0.0, 2.0 * SPACE_EXTENT, Ny)
    X, Y = np.meshgrid(x, y, indexing='xy')

    # -----------------------
    # Figure (server-side only) + front-end Image widget
    # -----------------------
    fig, ax = plt.subplots(figsize=(7.5, 6.2), dpi=DPI)
    im = ax.imshow(np.zeros((Ny, Nx)),
                   extent=[-SPACE_EXTENT, SPACE_EXTENT, 0.0, 2.0 * SPACE_EXTENT],
                   origin='lower', cmap=cmap_dd.value, vmin=-1.0, vmax=1.0, interpolation='bilinear')
    fig.colorbar(im, ax=ax, label='Wave Height')

    buoy_scatter = ax.scatter([], [], s=50, c='k', marker='o', label='Element locations')
    ax.legend(loc='upper right')
    ax.set_xlabel("X [m]"); ax.set_ylabel("Y [m]")
    ax.set_xlim(-SPACE_EXTENT, SPACE_EXTENT)
    ax.set_ylim(0.0, 2.0 * SPACE_EXTENT)
    fig.tight_layout()

    # Prevent the figure from being auto-captured
    plt.close(fig)

    # Front-end image updated with PNG bytes each frame
    img_widget = WImage(format='png', layout=Layout(width="750px"))

    def _render_png() -> bytes:
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=DPI)
        buf.seek(0)
        return buf.getvalue()

    # -----------------------
    # Core drawing routine (same push-to-PNG pattern as buoy sim)
    # -----------------------
    def _draw(frame_idx: int):
        lam = float(wavelength.value)
        N   = int(n_elements.value)
        ph  = float(phase_cycles.value)   # cycles per element (-0.5 .. +0.5)
        k   = 2.0 * np.pi / lam
        f   = WAVE_SPEED / lam
        omega = 2.0 * np.pi * f
        t = frame_idx * DT
        d = lam / 2.0

        # Element positions centered at x=0 along y=0
        positions = np.linspace(-(N - 1) / 2.0 * d, (N - 1) / 2.0 * d, N) if N > 0 else np.array([])
        phase_shifts = (np.arange(N) * (2.0 * np.pi * ph))  # convert cycles -> radians

        # Sum circular waves with progressive phase
        Z = np.zeros_like(X)
        for i, pos in enumerate(positions):
            R = np.hypot(X - pos, Y)
            Z += AMPLITUDE * np.sin(k * R - omega * t + phase_shifts[i])

        vmax = max(N * AMPLITUDE, 1.0)
        im.set_data(Z)
        im.set_clim(vmin=-vmax, vmax=vmax)

        deg_per_elem = ph * 360.0
        ax.set_title(f"Beam Steering— N={N}, λ={lam:.1f} m, d=λ/2, phase/elem={deg_per_elem:+.1f}°")

        if N > 0:
            buoy_scatter.set_offsets(np.c_[positions, np.zeros_like(positions)])
        else:
            buoy_scatter.set_offsets(np.empty((0, 2)))

        readout.value = (
            f"N = {N}  |  λ = {lam:.3g} m  |  phase/elem = {ph:+.2f} cycles ({deg_per_elem:+.1f}°)  |  "
            f"f = {f:.3g} Hz  |  v = {WAVE_SPEED:.3g} m/s"
        )

        # Push a fresh PNG to the frontend
        img_widget.value = _render_png()

    # -----------------------
    # Handlers
    # -----------------------
    def _reset(_):
        wavelength.value = 10.0
        n_elements.value = 6
        phase_cycles.value = 0.0
        play.value = 0
        t_slider.value = 0
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        lam = float(wavelength.value)
        N = int(n_elements.value)
        ph = float(phase_cycles.value)
        fname = f"outputs/beam_steering_lambda{lam:.1f}_N{N}_phase{ph:+.2f}_frame{int(t_slider.value)}.png"
        fig.savefig(fname, dpi=DPI, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # -----------------------
    # Wire up
    # -----------------------
    wavelength.observe(lambda _: _draw(t_slider.value), names='value')
    n_elements.observe(lambda _: _draw(t_slider.value), names='value')
    phase_cycles.observe(lambda _: _draw(t_slider.value), names='value')
    t_slider.observe(lambda ch: _draw(ch["new"]), names='value')
    def _update_colormap(_):
        im.set_cmap(cmap_dd.value)
        img_widget.value = _render_png()
    cmap_dd.observe(_update_colormap, names='value')

    # Initial frame
    _draw(t_slider.value)

    # Return composed UI (Voila-friendly)
    return VBox([controls, readout, img_widget])

# Build and display
steering_app = build_beam_steering_sim()
display(steering_app)


_Figure: Progressive phase shifts across elements steer the interference pattern; the main beam rotates left/right as phase/elem changes._

### 🧪 Try this
- Set phase/elem to +0.10 cycles and watch **the beam swing** right; set −0.10 to swing left.
- Increase the number of elements to tighten the mainlobe; decrease it to widen.

### **Buoys Rocking to Steer the Beam** 🌊📡

This short demo shows **how the buoys would move** (rocking up and down with tiny timing offsets) to send the wave energy in **different directions** ↗️ ↖️ ↘️ ↙️. Watch the blue dots: their **staggered bobbing** is the simple motion that produces the steered pattern in the field view.

- With **Phase/elem = 0**, all buoys move together → the wave goes **straight ahead**. ⬆️
- Adjust **Phase/elem** (positive or negative): each neighbor moves a split second earlier or later → the crest line **tilts**, and the beam swings **left or right**. ↔️  
- In other words, the **phase shift** is the delay between neighboring buoys, or how much out of sync their motion is. A larger shift means more delay, and the wavefront tilts more strongly. 🛟⏱️🌊 
- If you put enough buoys in the simulation, you’ll notice their motion starts to follow a **familiar pattern** along the array. Ring any bells? 🤔  

In [ ]:
def build_buoy_phase_offsets_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import FloatSlider, IntSlider, Play, IntSlider as IS, jslink, HBox, VBox, Layout, Output, Label, Button
    from IPython.display import display, clear_output

    # -----------------------
    # Fixed timeline & amplitude
    # -----------------------
    FRAMES = 200
    INTERVAL_MS = 50
    AMPLITUDE = 1.5  # fixed to match the simplified control set

    # -----------------------
    # Widgets (match beam-steering controls)
    # -----------------------
    n_elements = IntSlider(value=6, min=1, max=48, step=1,
                           description="Elements N", continuous_update=False)
    n_elements.style.description_width = 'initial'
    wavelength = FloatSlider(value=10.0, min=2.0, max=40.0, step=0.5,
                             description="Wavelength λ [m]", continuous_update=False)
    wavelength.style.description_width = 'initial'
    phase_cycles = FloatSlider(value=1/12, min=-0.5, max=0.5, step=0.005,
                               description="Phase/elem [cycles]", continuous_update=False)
    phase_cycles.style.description_width = 'initial'

    play = Play(value=0, min=0, max=FRAMES, step=1, interval=INTERVAL_MS)
    t_slider = IS(value=0, min=0, max=FRAMES, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')

    readout = Label()
    controls = VBox([
        HBox([n_elements, wavelength, phase_cycles],
             layout=Layout(align_items='center', column_gap='12px')),
        HBox([play, t_slider, reset_btn, save_btn],
             layout=Layout(align_items='center', column_gap='12px'))
    ])

    # -----------------------
    # Output / Figure
    # -----------------------
    out = Output()
    fig, ax = plt.subplots(figsize=(8, 4))
    plt.close(fig)  # avoid static capture in Voilà/Jupyter

    # Mutable state
    state = {
        "x_positions": None,
        "scatter": None,
        "guides": [],
        "last_N": None,
        "last_d": None,
    }

    def _rebuild_scene(N: int, lam: float):
        ax.cla()
        d = lam / 2.0
        x_positions = np.linspace(-((N - 1) / 2) * d, ((N - 1) / 2) * d, N)

        margin = 1.5 * d
        ax.set_xlim(x_positions.min() - margin, x_positions.max() + margin)
        ax.set_ylim(-1.2 * AMPLITUDE, 1.2 * AMPLITUDE)
        ax.set_xlabel("Buoy Position x [m]")
        ax.set_ylabel("Buoy Displacement")
        ax.set_title("Buoy Array Rocking with Phase Offsets (d = λ/2)")

        # Vertical guide lines
        guides = []
        for x in x_positions:
            line, = ax.plot([x, x], [-2, 2], 'k--', linewidth=0.6)
            guides.append(line)

        # Scatter for buoys
        scatter, = ax.plot([], [], 'bo', markersize=8)

        state["x_positions"] = x_positions
        state["guides"] = guides
        state["scatter"] = scatter
        state["last_N"] = N
        state["last_d"] = d

    def _draw(frame_idx: int):
        N = int(n_elements.value)
        lam = float(wavelength.value)
        ph_cyc = float(phase_cycles.value)  # cycles per element
        ph_rad = 2.0 * np.pi * ph_cyc      # radians per element

        # Rebuild if geometry changed
        d = lam / 2.0
        if state["last_N"] is None or state["last_d"] is None or state["last_N"] != N or abs(state["last_d"] - d) > 1e-12:
            _rebuild_scene(N, lam)

        x = state["x_positions"]

        # Time progression (0..2π over FRAMES)
        t = frame_idx * 2 * np.pi / FRAMES

        # Phase offsets across buoys: 0, φ, 2φ, ...
        n_idx = np.arange(N)
        y = AMPLITUDE * np.sin(t + n_idx * ph_rad)

        # Update visuals
        state["scatter"].set_data(x, y)
        ax.set_ylim(-1.2 * AMPLITUDE, 1.2 * AMPLITUDE)

        deg_per_elem = ph_cyc * 360.0
        readout.value = (f"N={N}, λ={lam:.2f} m, d=λ/2={d:.2f} m | "
                         f"phase/elem={ph_cyc:+.3f} cycles ({deg_per_elem:+.1f}°)")

        with out:
            clear_output(wait=True)
            display(fig)

    # Handlers
    def _reset(_):
        n_elements.value = 6
        wavelength.value = 10.0
        phase_cycles.value = 1/12
        play.value = 0
        t_slider.value = 0
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        lam = float(wavelength.value)
        N = int(n_elements.value)
        ph = float(phase_cycles.value)
        fname = f"outputs/buoy_phase_offsets_N{N}_lam{lam:.1f}_phase{ph:+.3f}_frame{int(t_slider.value)}.png"
        fig.savefig(fname, dpi=150, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # Wire up interactions
    for w in (n_elements, wavelength, phase_cycles):
        w.observe(lambda ch: _draw(t_slider.value), names='value')
    t_slider.observe(lambda ch: _draw(ch["new"]), names='value')

    # Initial render
    _draw(t_slider.value)

    return VBox([controls, readout, out])

# Build & show
buoy_phase_app = build_buoy_phase_offsets_sim()
display(buoy_phase_app)


_Figure: Element motions with phase offsets; staggered bobbing corresponds to a tilted composite wavefront and a steered beam._

### 🧪 Try this
- Zero phase/elem to see all buoys move together (beam straight ahead).
- Increase phase/elem magnitude to exaggerate the tilt and steering.
- Add elements to observe the phase ramp across a longer array.


### **Why Does the Beam Change Direction?** 📡🔄  

To analyze why the direction of the beam pattern shifts, let’s return to our best friends—the **crests and troughs** of the wavefronts coming from each buoy. 🛟🌊  

Since the buoys **start their cycles at different phases**, the crests (and troughs) from each buoy have **expanded different distances**.  

Now, if we look at the places where these wavefronts **align to combine in the same phase**, something fascinating happens:  
- The alignment is **tilted** with respect to the array! 📏
- The larger the **phase difference** between the elements, the more the **wavefront is tilted**. 📈
- **Voilà!** By carefully controlling the **phase shift** between the buoys, we can **steer the beam in any direction we want!** 📡✨  

---

### **The Most Incredible Part?**  

🔹 This is all achieved **without physically turning** the buoy array!   

Once again, just replace the buoys with antenna elements, and you’ll see exactly **how a phased array radar antenna works!** 📡  In radar, the **phase of different antenna elements** can be adjusted **very precisely** and switched **very rapidly**. ⚡  Thus, we can change the **direction of illumination**(and reception, as we'll learn soon) in just a matter of tiny fractions of a second! ⏳✨    


### **Crest–Trough View: Steering in Ripples** 🌊🔭

This animation shows **expanding rings** from each buoy: **solid = crests**, **dashed = troughs**.  
By giving neighboring buoys **phase offsets** (see **Phase/elem**), their rings no longer line up symmetrically—the combined wavefront **leans**, and energy heads **left or right**.

- Where **many crests meet**, you’ll see **bright, focused regions** (constructive interference). 🌊➕🌊
- Where **crests overlap troughs**, the pattern **cancels** (destructive interference).  🌊➖🌊
- Try changing **Phase/elem** to swing the focus, and adjust **Elements N** or **Wavelength λ** to see how the geometry reshapes the pattern.

In [ ]:
def build_crest_trough_phase_shift_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import FloatSlider, IntSlider, Play, IntSlider as IS, jslink, HBox, VBox, Layout, Output, Label, Button
    from IPython.display import display, clear_output

    # -----------------------
    # Fixed viewing parameters
    # -----------------------
    SPACE_EXTENT = 150.0      # meters; fixed x in [-E, E], y in [0, 2E]
    FRAMES = 200              # timeline length
    INTERVAL_MS = 80          # Play interval (ms)
    CIRCLE_POINTS = 250       # circle resolution for rings

    # -----------------------
    # Widgets
    # -----------------------
    phase_cycles = FloatSlider(
        value=0, min=-0.5, max=0.5, step=0.01,
        description="Phase/elem [cycles]", continuous_update=False
    )
    phase_cycles.style.description_width = 'initial'

    wavelength = FloatSlider(
        value=20.0, min=5.0, max=40.0, step=0.5,
        description="Wavelength λ [m]", continuous_update=False
    )
    wavelength.style.description_width = 'initial'

    n_elements = IntSlider(
        value=4, min=1, max=16, step=1,
        description="Elements N", continuous_update=False
    )
    n_elements.style.description_width = 'initial'

    n_rings = IntSlider(
        value=10, min=3, max=30, step=1,
        description="Rings shown", continuous_update=False
    )
    n_rings.style.description_width = 'initial'

    play = Play(value=0, min=0, max=FRAMES, step=1, interval=INTERVAL_MS)
    t_slider = IS(value=0, min=0, max=FRAMES, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')

    readout = Label()

    # Two rows of controls
    row1 = HBox([phase_cycles, wavelength], layout=Layout(align_items='center', column_gap='12px'))
    row2 = HBox([n_elements, n_rings, play, t_slider, reset_btn, save_btn], layout=Layout(align_items='center', column_gap='12px'))
    controls = VBox([row1, row2])

    # -----------------------
    # Output / Figure
    # -----------------------
    out = Output()
    fig, ax = plt.subplots(figsize=(8, 6))
    plt.close(fig)  # prevent static snapshot in notebooks/Voilà

    # State
    state = {
        "wave_lines": [],
        "trough_lines": [],
        "positions": None,
        "theta": np.linspace(0, 2*np.pi, CIRCLE_POINTS),
        "lam": None,
        "N": None,
        "num_rings": None,
        "initial_crest_locations": None,
        "phase_cyc": None,
        # New: beam & null visualizers
        "beam_line": None,
        "null_lines": [],
    }

    def _make_initial_crest_locations(lam: float, num_rings: int):
        # Rings spaced by λ, shifted so a crest starts near the sources at t=0
        locs = np.linspace(0, (num_rings - 1) * lam, num_rings)
        if num_rings >= 2:
            locs = locs - locs[-2]
        return locs

    def _ensure_beam_artists():
        """Create (or re-create) artists for main beam and first nulls."""
        # Remove old if present
        if state["beam_line"] is not None:
            try:
                state["beam_line"].remove()
            except Exception:
                pass
        for ln in state["null_lines"]:
            try:
                ln.remove()
            except Exception:
                pass
        state["null_lines"] = []

        # Main beam: thicker
        state["beam_line"], = ax.plot([], [], linestyle='-', linewidth=2.5, color='green', label="Main Beam")

        # Two first nulls: thinner
        n1, = ax.plot([], [], linestyle='--', linewidth=1.2, color='gray', label="First Nulls")
        n2, = ax.plot([], [], linestyle='--', linewidth=1.2, color='gray')
        state["null_lines"] = [n1, n2]

        # Refresh legend (keep existing entries)
        handles, labels = ax.get_legend_handles_labels()
        # De-duplicate while preserving order
        seen = set()
        uniq = [(h, l) for h, l in zip(handles, labels) if (l not in seen and not seen.add(l))]
        ax.legend(*zip(*uniq), loc='upper right', frameon=True)

    def _rebuild_scene(N: int, lam: float, num_rings: int):
        ax.cla()
        ax.set_xlim(-SPACE_EXTENT, SPACE_EXTENT)
        ax.set_ylim(0.0, 1.5 * SPACE_EXTENT)
        ax.set_aspect('equal', adjustable='box')
        ax.set_xlabel("X [m]")
        ax.set_ylabel("Y [m]")
        ax.set_title(f"Expanding Wavefronts from {N}-Element Array (crest/trough view)")
        ax.grid(False)

        # Element spacing (λ/2) and positions along x at y=0
        d = lam / 2.0
        positions = np.linspace(-(N - 1) / 2.0 * d, (N - 1) / 2.0 * d, N)

        # Create empty ring artists
        state["wave_lines"] = []
        state["trough_lines"] = []
        for _ in range(N * num_rings):
            crest_line, = ax.plot([], [], 'r-', linewidth=1.2)   # solid = crests
            trough_line, = ax.plot([], [], 'b--', linewidth=1.2) # dashed = troughs
            state["wave_lines"].append(crest_line)
            state["trough_lines"].append(trough_line)

        # Legend with dummy handles (before beam/nulls so we can dedupe after)
        ax.plot([], [], 'r-', linewidth=1.5, label="Wave Crest")
        ax.plot([], [], 'b--', linewidth=1.5, label="Wave Trough")

        # Plot buoy locations
        ax.scatter(positions, np.zeros_like(positions), label="Buoy Locations",
                   color="black", s=50, marker='o')

        # Create beam + null artists
        _ensure_beam_artists()

        # Store scene params
        state["positions"] = positions
        state["lam"] = lam
        state["N"] = N
        state["num_rings"] = num_rings
        state["initial_crest_locations"] = _make_initial_crest_locations(lam, num_rings)

    def _update_beam_and_nulls(lam: float, N: int, ph_cyc: float):
        """
        Compute and draw the main-beam direction and first two nulls.
        Angles are measured from +x (to match the circle param), with y ≥ 0 in view.
        """
        # Geometry / steering parameters
        d = lam / 2.0
        k = 2.0 * np.pi / lam
        kd = k * d                 # = π for d=λ/2
        beta = 2.0 * np.pi * ph_cyc

        # Helper: set a ray (from origin) if |cosθ|≤1; otherwise hide it.
        def _set_ray(line_artist, cos_theta):
            if line_artist is None:
                return
            if np.abs(cos_theta) <= 1.0:
                theta = np.arccos(np.clip(cos_theta, -1.0, 1.0))
                # Build a long segment; clipping will keep it inside axes
                L = 2.5 * SPACE_EXTENT
                x = np.array([0.0, L * np.cos(theta)])
                y = np.array([0.0, L * np.sin(theta)])
                line_artist.set_data(x, y)
                line_artist.set_visible(True)
            else:
                line_artist.set_data([], [])
                line_artist.set_visible(False)

        # Main lobe (ψ = 0 ⇒ kd cosθ = β)
        cos_th0 = beta / kd if kd != 0 else np.nan
        _set_ray(state["beam_line"], cos_th0)

        # First nulls: kd cosθ - β = ± 2π/N
        if N and kd != 0:
            delta = 2.0 * np.pi / N
            cos_n1 = (beta + delta) / kd
            cos_n2 = (beta - delta) / kd
            _set_ray(state["null_lines"][0], cos_n1)
            _set_ray(state["null_lines"][1], cos_n2)
        else:
            for ln in state["null_lines"]:
                ln.set_data([], [])
                ln.set_visible(False)

    def _draw(frame_idx: int):
        lam = float(wavelength.value)
        N = int(n_elements.value)
        num_rings = int(n_rings.value)
        ph_cyc = float(phase_cycles.value)  # cycles per element (−0.5..+0.5)

        # Rebuild if geometry changed
        if (state["lam"] is None or abs(state["lam"] - lam) > 1e-12 or
            state["N"] != N or state["num_rings"] != num_rings):
            _rebuild_scene(N, lam, num_rings)

        positions = state["positions"]
        theta = state["theta"]
        init_locs = state["initial_crest_locations"]

        # Propagation speed tied to fixed extent
        distance_per_frame = SPACE_EXTENT / FRAMES

        # Phase-induced radial shift per element (cycles × λ/2)
        element_shifts = np.arange(N) * ph_cyc * lam / 2.0

        # Update rings
        for i, pos in enumerate(positions):
            for j in range(num_rings):
                idx = i * num_rings + j
                radius_crest = (frame_idx - j) * distance_per_frame + init_locs[j] + element_shifts[i]
                radius_trough = radius_crest - lam / 2.0

                if radius_crest > 0:
                    x_crest = pos + radius_crest * np.cos(theta)
                    y_crest = radius_crest * np.sin(theta)
                    state["wave_lines"][idx].set_data(x_crest, y_crest)
                else:
                    state["wave_lines"][idx].set_data([], [])

                if radius_trough > 0:
                    x_trough = pos + radius_trough * np.cos(theta)
                    y_trough = radius_trough * np.sin(theta)
                    state["trough_lines"][idx].set_data(x_trough, y_trough)
                else:
                    state["trough_lines"][idx].set_data([], [])

        # Update beam & null overlays (depends on N, λ, phase)
        _update_beam_and_nulls(lam, N, ph_cyc)

        deg_per_elem = ph_cyc * 360.0
        d = lam / 2.0
        readout.value = (f"N = {N},  λ = {lam:.2f} m,  d = λ/2 = {d:.2f} m  |  "
                         f"phase per element = {ph_cyc:+.2f} cycles ({deg_per_elem:+.1f}°)  |  "
                         f"rings = {num_rings}")

        with out:
            clear_output(wait=True)
            display(fig)

    # Handlers
    def _reset(_):
        phase_cycles.value = 0.0
        wavelength.value = 20.0
        n_elements.value = 4
        n_rings.value = 10
        play.value = 0
        t_slider.value = 0
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        lam = float(wavelength.value)
        N = int(n_elements.value)
        ph = float(phase_cycles.value)
        R = int(n_rings.value)
        fname = f"outputs/crest_trough_phase_lam{lam:.1f}_N{N}_phase{ph:+.2f}_rings{R}_frame{int(t_slider.value)}.png"
        fig.savefig(fname, dpi=150, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # Wire up interactions
    for w in (phase_cycles, wavelength, n_elements, n_rings):
        w.observe(lambda ch: _draw(t_slider.value), names='value')
    t_slider.observe(lambda ch: _draw(ch["new"]), names='value')

    # Initial render
    _draw(t_slider.value)

    return VBox([controls, readout, out])

# Build & show
crest_trough_app = build_crest_trough_phase_shift_sim()
display(crest_trough_app)


_Figure: Crest (solid) and trough (dashed) rings with steering overlays; green ray shows main beam, gray rays mark first nulls._

### 🧪 Try this
- Sweep phase/elem to swing the green main-beam ray; verify nulls track it.
- Increase the number of elements to move first nulls inward (narrower beam); decrease it to widen.


## 📌 Summary

- **Phased-array beam steering** is achieved with **progressive phase shifts** across elements—**no mechanical antenna motion** required.  
- The **amount of phase shift between elements** sets the **beam angle**. 
- With **fixed element spacing**, adding more elements makes the **main beam narrower**.  
